In [14]:
import os
import openslide
import pandas as pd
import numpy as np
import matplotlib
import matplotlib.pyplot as plt
import PIL
from IPython.display import Image, display
import skimage.io
import tensorflow as tf
from tensorflow import keras
import tensorflow_addons as tfa

In [15]:
#Sökväg till mappen där de processeade bilderna finns
data_dir = r'D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages'
#ISUP-graderingarna för bilderna
labels = pd.read_csv(r'D:\PhDMain\Courses\DeepLearning\PANDAS\train.csv')

In [16]:
#Om vi skall använda N=16 eller N=100, dvs. de stora eller små bilderna. För denna kod använder vi bara de mindre bilderna. De 
#större används i en annan kod.
SIZE='small'
if SIZE=='small':
    DIM=512
    batch_size=32
elif SIZE=='large':
    DIM=1280
    batch_size=5

#Antal bilder i varje kategori
no_train=7962
no_valid=996
no_test=996
#Antal ISUP-graderingar, alla heltal i intervallet [0,5]
no_classes=6

In [17]:
#All data tar upp väldigt mycket RAM-minne; vi använder därför en generator som kan läsa in de bilderna från hårddisken
#som behövs för en batch åt gången. 
#Som grund för denna kod har jag använt mig av följande:
#https://medium.com/datadriveninvestor/keras-training-on-large-datasets-3e9d9dbc09d4
class DataGenerator(keras.utils.Sequence) :
    #Detta är vår konstruktor; den tar sökvägen till mappen vi vill använda, ISUP-graderingarna, batch-storlek samt om vi
    #vill använda de små eller de stora bilderna. 
  def __init__(self, path, labels, batch_size,image_size):
    self.path=path
    self.file_names = [name for name in os.listdir(path)]
    label_dict=dict(labels[['image_id','isup_grade']].values)
    self.labels = [label_dict[name[:-4]] for name in self.file_names]
    self.batch_size = batch_size
    self.image_size = image_size

    #Vi behöver definera [] och len() för att generator skall fungera. len() returner antalet batcher som behövs för en full
    #träningssepok.
  def __len__(self):
    return (np.ceil(len(self.file_names) / float(self.batch_size))).astype(np.int)

    #[i] returnerar batch i som två numpy-arrays; den första är bilderna, den andra är ISUP-graderingarna.
  def __getitem__(self, idx):
    batch_x = self.file_names[idx * self.batch_size : (idx+1) * self.batch_size]
    batch_y = self.labels[idx * self.batch_size : (idx+1) * self.batch_size]
    
    img_list=list()
    for file_name in batch_x:
        data=np.load(os.path.join(self.path,file_name))
        img_list.append(data[self.image_size])
        data.close()
    return np.array(img_list), np.array(batch_y)


In [18]:
#Vi sätter upp generatorer för både träningss- och valideringssettet.
train_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\TrainSet', labels, batch_size,SIZE)
valid_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\ValidationSet', labels, batch_size,SIZE)

In [38]:
#Vi laddar ner ResNet50V2 med tillhörande vikter via träning på ImageNet. Vi sätter också upp en
#dictionary för lager namnen så vi enkelt kan med lager från ResNet via lagernamnen.
ResNet50 = keras.applications.ResNet50V2(weights="imagenet",include_top=False,input_shape=[DIM,DIM,3])
LayerDict = dict([(layer.name, layer) for layer in ResNet50.layers])
ResNet50.summary()

Model: "resnet50v2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
_________________________________________________________________________________________

In [37]:
#Vi fryser alla lager i ResNet för träning.
for i in range(len(ResNet50.layers)):
    ResNet50.layers[i].trainable = False

In [22]:
#Definitionen av den första modellen som vi testar; sex faltningslager, två åt gången, separerade av pooling-lager och 
#batch normalization. Sist har vi ett softmax-lager med 6 noder.
NM = LayerDict['post_relu'].output
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=512, kernel_size=(2,2), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=512, kernel_size=(2,2), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Flatten()(NM)
NM = keras.layers.Dense(no_classes, activation="softmax")(NM)
Model1 = tf.keras.models.Model(inputs=ResNet50.input, outputs=NM)
Model1.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [23]:
#Vi använder konsekvent sparse categorical crossentropy som loss-funktion, som passar väl
#med kategoriseringsproblemet.Adam som vår optimeringsmotor för att snabba upp processen något.  
Model1.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc"),tfa.metrics.CohenKappa(weightage='quadratic',
                                                                                                 num_classes=6,
                                                                                                 sparse_labels=True)])

In [24]:
#Vi tränar till det att vi inte har sett en förbättring i kvadratiska kappa-värdet på 10 epoker för valideringssettet, upp till maximalt 30 epoker.
#Vi tar därefter vikterna från en epok som gav bästa kvadratiska kappa-värde för valideringssettet.
Model1.fit_generator(generator=train_generator,
                   steps_per_epoch = int(no_train // batch_size),
                   epochs = 30,
                   verbose = 1,
                   validation_data = valid_generator,
                   validation_steps = int(no_valid // batch_size),
                   callbacks=keras.callbacks.EarlyStopping(monitor='val_cohen_kappa',mode='max',patience=10, restore_best_weights=True))

Epoch 1/30
248/248 [==============================] - 347s 1s/step - loss: 1.8532 - acc: 0.3180 - cohen_kappa: 0.4191 - val_loss: 2.3193 - val_acc: 0.3165 - val_cohen_kappa: 0.2025
Epoch 2/30
248/248 [==============================] - 94s 380ms/step - loss: 1.5668 - acc: 0.3871 - cohen_kappa: 0.5326 - val_loss: 1.6218 - val_acc: 0.4294 - val_cohen_kappa: 0.5625
Epoch 3/30
248/248 [==============================] - 94s 381ms/step - loss: 1.4656 - acc: 0.4385 - cohen_kappa: 0.5698 - val_loss: 1.5728 - val_acc: 0.4254 - val_cohen_kappa: 0.3826
Epoch 4/30
248/248 [==============================] - 96s 386ms/step - loss: 1.3966 - acc: 0.4550 - cohen_kappa: 0.5939 - val_loss: 1.6155 - val_acc: 0.3810 - val_cohen_kappa: 0.5560
Epoch 5/30
248/248 [==============================] - 93s 376ms/step - loss: 1.3444 - acc: 0.4729 - cohen_kappa: 0.6276 - val_loss: 1.5894 - val_acc: 0.4325 - val_cohen_kappa: 0.4838
Epoch 6/30
248/248 [==============================] - 99s 398ms/step - loss: 1.2999 - a

In [26]:
#Andra modellen; färre faltningslager, ett fully connected dense-layer med 100 noder samt ett dropout-lager med 0.5 fraction.
NM = LayerDict['post_relu'].output
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Flatten()(NM)
NM = keras.layers.Dense(100, activation="relu")(NM)
NM = keras.layers.Dropout(0.5)(NM)
NM = keras.layers.Dense(no_classes, activation="softmax")(NM)
Model2 = tf.keras.models.Model(inputs=ResNet50.input, outputs=NM)
Model2.summary()


Model: "model_3"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_2[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

In [29]:
#Träning av andra modellen.
Model2.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc"),tfa.metrics.CohenKappa(weightage='quadratic',
                                                                                                 num_classes=6,
                                                                                                 sparse_labels=True)])

Model2.fit_generator(generator=train_generator,
                   steps_per_epoch = int(no_train // batch_size),
                   epochs = 30,
                   verbose = 1,
                   validation_data = valid_generator,
                   validation_steps = int(no_valid // batch_size),
                   callbacks=keras.callbacks.EarlyStopping(monitor='val_cohen_kappa',mode='max',patience=10, restore_best_weights=True))

Epoch 1/30
248/248 [==============================] - 85s 342ms/step - loss: 1.7338 - acc: 0.2692 - cohen_kappa: 0.0244 - val_loss: 1.5929 - val_acc: 0.2853 - val_cohen_kappa: 0.0150
Epoch 2/30
248/248 [==============================] - 86s 345ms/step - loss: 1.6134 - acc: 0.2938 - cohen_kappa: 0.0931 - val_loss: 1.5853 - val_acc: 0.3085 - val_cohen_kappa: 0.1617
Epoch 3/30
248/248 [==============================] - 85s 344ms/step - loss: 1.5889 - acc: 0.3156 - cohen_kappa: 0.2375 - val_loss: 1.4996 - val_acc: 0.3629 - val_cohen_kappa: 0.4373
Epoch 4/30
248/248 [==============================] - 87s 350ms/step - loss: 1.5436 - acc: 0.3395 - cohen_kappa: 0.3759 - val_loss: 1.7114 - val_acc: 0.3609 - val_cohen_kappa: 0.4719
Epoch 5/30
248/248 [==============================] - 84s 340ms/step - loss: 1.5244 - acc: 0.3541 - cohen_kappa: 0.3841 - val_loss: 1.5821 - val_acc: 0.3841 - val_cohen_kappa: 0.4997
Epoch 6/30
248/248 [==============================] - 84s 340ms/step - loss: 1.4928 -

In [39]:
for i in range(142):
    ResNet50.layers[i].trainable = False
#Definition av tredje modellen. Vi tar med några färre lager från ResNet denna gång.
NM = LayerDict['conv4_block5_out'].output
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Flatten()(NM)
NM = keras.layers.Dense(100, activation="relu")(NM)
NM = keras.layers.Dropout(0.5)(NM)
NM = keras.layers.Dense(no_classes, activation="softmax")(NM)
Model3 = tf.keras.models.Model(inputs=ResNet50.input, outputs=NM)

Model3.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc"),tfa.metrics.CohenKappa(weightage='quadratic',
                                                                                                 num_classes=6,
                                                                                                 sparse_labels=True)])

Model3.fit_generator(generator=train_generator,
                   steps_per_epoch = int(no_train // batch_size),
                   epochs = 30,
                   verbose = 1,
                   validation_data = valid_generator,
                   validation_steps = int(no_valid // batch_size),
                   callbacks=keras.callbacks.EarlyStopping(monitor='val_cohen_kappa',mode='max',patience=10, restore_best_weights=True))


Epoch 1/30
248/248 [==============================] - 85s 342ms/step - loss: 1.8500 - acc: 0.2714 - cohen_kappa: -0.0049 - val_loss: 1.7455 - val_acc: 0.2873 - val_cohen_kappa: -2.6977e-04
Epoch 2/30
248/248 [==============================] - 85s 342ms/step - loss: 1.7314 - acc: 0.2851 - cohen_kappa: -1.1921e-07 - val_loss: 1.7195 - val_acc: 0.2863 - val_cohen_kappa: 0.0000e+00
Epoch 3/30
248/248 [==============================] - 85s 342ms/step - loss: 1.7137 - acc: 0.2859 - cohen_kappa: 1.7881e-07 - val_loss: 1.7089 - val_acc: 0.2863 - val_cohen_kappa: 0.0000e+00
Epoch 4/30
248/248 [==============================] - 85s 341ms/step - loss: 1.7070 - acc: 0.2860 - cohen_kappa: -1.1921e-07 - val_loss: 1.7050 - val_acc: 0.2863 - val_cohen_kappa: 0.0000e+00
Epoch 5/30
248/248 [==============================] - 84s 341ms/step - loss: 1.7052 - acc: 0.2858 - cohen_kappa: -1.1921e-07 - val_loss: 1.7038 - val_acc: 0.2863 - val_cohen_kappa: 0.0000e+00
Epoch 6/30
248/248 [========================

In [42]:
#Då tredje modellen gav mycket dåliga resultat går vi tillbaka till att ta med hela ResNet, fast med en större batch_size.
batch_size=64
for i in range(len(ResNet50.layers)):
    ResNet50.layers[i].trainable = False
NM = LayerDict['post_relu'].output
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Flatten()(NM)
NM = keras.layers.Dense(100, activation="relu")(NM)
NM = keras.layers.Dropout(0.5)(NM)
NM = keras.layers.Dense(no_classes, activation="softmax")(NM)
Model4 = tf.keras.models.Model(inputs=ResNet50.input, outputs=NM)
Model4.summary()

train_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\TrainSet', labels, batch_size,SIZE)
valid_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\ValidationSet', labels, batch_size,SIZE)

Model4.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc"),tfa.metrics.CohenKappa(weightage='quadratic',
                                                                                                 num_classes=6,
                                                                                                 sparse_labels=True)])

Model4.fit_generator(generator=train_generator,
                   steps_per_epoch = int(no_train // batch_size),
                   epochs = 30,
                   verbose = 1,
                   validation_data = valid_generator,
                   validation_steps = int(no_valid // batch_size),
                   callbacks=keras.callbacks.EarlyStopping(monitor='val_cohen_kappa',mode='max',patience=10, restore_best_weights=True))


Model: "model_7"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

Epoch 1/30
124/124 [==============================] - 85s 685ms/step - loss: 1.6854 - acc: 0.2949 - cohen_kappa: 0.1833 - val_loss: 1.5606 - val_acc: 0.3031 - val_cohen_kappa: 0.1061
Epoch 2/30
124/124 [==============================] - 84s 679ms/step - loss: 1.5872 - acc: 0.3025 - cohen_kappa: 0.1508 - val_loss: 1.5319 - val_acc: 0.3375 - val_cohen_kappa: 0.1728
Epoch 3/30
124/124 [==============================] - 84s 679ms/step - loss: 1.5592 - acc: 0.3164 - cohen_kappa: 0.2683 - val_loss: 1.5783 - val_acc: 0.3604 - val_cohen_kappa: 0.5189
Epoch 4/30
124/124 [==============================] - 84s 681ms/step - loss: 1.5160 - acc: 0.3572 - cohen_kappa: 0.4181 - val_loss: 1.5314 - val_acc: 0.3833 - val_cohen_kappa: 0.4715
Epoch 5/30
124/124 [==============================] - 85s 685ms/step - loss: 1.4771 - acc: 0.3725 - cohen_kappa: 0.4643 - val_loss: 1.5903 - val_acc: 0.4167 - val_cohen_kappa: 0.5781
Epoch 6/30
124/124 [==============================] - 84s 680ms/step - loss: 1.4327 -

In [43]:
#Tillbaka till vanliga batch-storlek då den större batch-storleken inte gav något påtagligt bättre resultat. 
#Vi prövar med att lägga till ett par fully connected dense layers med 512 noder vardera.
batch_size=32

NM = LayerDict['post_relu'].output
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(4,4), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.Conv2D(filters=256, kernel_size=(3,3), activation='relu',padding="same")(NM)
NM = keras.layers.MaxPool2D()(NM)
NM = keras.layers.BatchNormalization(axis=3)(NM)
NM = keras.layers.Flatten()(NM)
NM = keras.layers.Dense(512, activation="relu")(NM)
NM = keras.layers.Dropout(0.5)(NM)
NM = keras.layers.Dense(512, activation="relu")(NM)
NM = keras.layers.Dropout(0.5)(NM)
NM = keras.layers.Dense(no_classes, activation="softmax")(NM)
Model5 = tf.keras.models.Model(inputs=ResNet50.input, outputs=NM)
Model5.summary()

train_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\TrainSet', labels, batch_size,SIZE)
valid_generator = DataGenerator('D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\ValidationSet', labels, batch_size,SIZE)

Model5.compile(loss="sparse_categorical_crossentropy", optimizer="adam",
              metrics=[keras.metrics.SparseCategoricalAccuracy(name="acc"),tfa.metrics.CohenKappa(weightage='quadratic',
                                                                                                 num_classes=6,
                                                                                                 sparse_labels=True)])

Model5.fit_generator(generator=train_generator,
                   steps_per_epoch = int(no_train // batch_size),
                   epochs = 30,
                   verbose = 1,
                   validation_data = valid_generator,
                   validation_steps = int(no_valid // batch_size),
                   callbacks=keras.callbacks.EarlyStopping(monitor='val_cohen_kappa',mode='max',patience=10, restore_best_weights=True))


Model: "model_8"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_3 (InputLayer)            [(None, 512, 512, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 518, 518, 3)  0           input_3[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 256, 256, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 258, 258, 64) 0           conv1_conv[0][0]                 
____________________________________________________________________________________________

Epoch 1/30
248/248 [==============================] - 84s 340ms/step - loss: 1.8868 - acc: 0.2769 - cohen_kappa: 0.1956 - val_loss: 1.7168 - val_acc: 0.2863 - val_cohen_kappa: 0.0122
Epoch 2/30
248/248 [==============================] - 84s 340ms/step - loss: 1.6146 - acc: 0.3083 - cohen_kappa: 0.1699 - val_loss: 1.5977 - val_acc: 0.3286 - val_cohen_kappa: 0.1422
Epoch 3/30
248/248 [==============================] - 84s 339ms/step - loss: 1.6003 - acc: 0.3087 - cohen_kappa: 0.1338 - val_loss: 1.5433 - val_acc: 0.3438 - val_cohen_kappa: 0.1686
Epoch 4/30
248/248 [==============================] - 85s 341ms/step - loss: 1.5562 - acc: 0.3185 - cohen_kappa: 0.1631 - val_loss: 1.5348 - val_acc: 0.3216 - val_cohen_kappa: 0.1750
Epoch 5/30
248/248 [==============================] - 84s 340ms/step - loss: 1.5387 - acc: 0.3420 - cohen_kappa: 0.3997 - val_loss: 1.5781 - val_acc: 0.3266 - val_cohen_kappa: 0.4355
Epoch 6/30
248/248 [==============================] - 85s 343ms/step - loss: 1.5046 -

In [44]:
#Den andra modellen med avseende på valideringssettet gav högst kvadratiskt kappa; vi utvärderar därför detta result mot
#testsettet.

test_label_dict=dict(labels[['image_id','isup_grade']].values)
test_path='D:\PhDMain\Courses\DeepLearning\PANDAS\ProcessedImages\TestSet'

test_file_names = [name for name in os.listdir(test_path)]
test_x=np.ndarray(shape=(len(test_file_names),DIM,DIM,3),dtype=np.dtype('float32'))
test_y=np.ndarray(shape=len(test_file_names))
for i in range(len(test_file_names)):
    name=test_file_names[i]
    data=np.load(os.path.join(test_path,name))
    test_x[i,:,:,:]=data[SIZE]
    test_y[i]=test_label_dict[name[:-4]]

In [50]:
Model2.evaluate(test_x,test_y)

32/32 [==============================] - 8s 245ms/step - loss: 1.5203 - acc: 0.3956 - cohen_kappa: 0.6240


[1.5203049182891846, 0.39558231830596924, 0.6240477561950684]

In [ ]:
#Det kvadratiska kappa-värdet på dryga 0.62 för testsettet visar att vi är klart bättre än slumpen. Accuracy-värdet på dryga
#0.39 visar detsamma, även om vi alltså har fel oftare än rätt. Det högre kappa-värdet visar dock att vi, när vi har fel, ändå tämligen
#ofta är nära det sanna värdet.